In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

In [30]:
def score_dataset(X,y, model = XGBRegressor()):
    for cols in X.select_dtypes(["category","object"]):
        X[cols],_ = X[cols].factorize()
    score = cross_val_score(
        model,X,y,cv = 5,
        scoring="neg_mean_squared_log_error"
    )
    score = -1*score.mean()
    score = np.sqrt(score)
    return score

In [ ]:
df  = pd.read_csv("./ames.csv")
X = df.copy()
y = X.pop("SalePrice")

In [ ]:
X_1 = pd.DataFrame()

X_1["LivLotRatio"] = df['GrLivArea']/df['LotArea']
X_1["Spaciousness"] = (df['FirstFlrSF']+ df['SecondFlrSF'])/df['TotRmsAbvGrd']
X_1['TotalOutsideSF'] = df['WoodDeckSF'] + df['Threeseasonporch'] + df['OpenPorchSF'] + df['EnclosedPorch'] + df['Threeseasonporch'] + df['ScreenPorch']

X_1.head()

,LivLotRatio,Spaciousness,TotalOutsideSF
0,0.052125,236.571429,272.0
1,0.077095,179.200000,260.0
2,0.093152,221.500000,429.0
3,0.189068,263.750000,0.0
4,0.117787,271.500000,246.0


In [12]:
# X_new = pd.get_dummies(df['MSZoning'], prefix="Cat")
# X_new = X_new.mul(df['MSSubClass'], axis = 0)
# X = X.join(X_new)

# X.head()

In [14]:
X_2 = pd.get_dummies(df['BldgType'], prefix='Bldg')
X_2.head()

,Bldg_Duplex,Bldg_OneFam,Bldg_Twnhs,Bldg_TwnhsE,Bldg_TwoFmCon
0,False,True,False,False,False
1,False,True,False,False,False
2,False,True,False,False,False
3,False,True,False,False,False
4,False,True,False,False,False


In [15]:
X_2 = X_2.mul(df['GrLivArea'], axis=0)

In [16]:
X_2.head()

,Bldg_Duplex,Bldg_OneFam,Bldg_Twnhs,Bldg_TwnhsE,Bldg_TwoFmCon
0,0.0,1656.0,0.0,0.0,0.0
1,0.0,896.0,0.0,0.0,0.0
2,0.0,1329.0,0.0,0.0,0.0
3,0.0,2110.0,0.0,0.0,0.0
4,0.0,1629.0,0.0,0.0,0.0


In [17]:
X_3 = pd.DataFrame()

X_3['PorchTypes'] = df[[
      'WoodDeckSF','OpenPorchSF',
    'EnclosedPorch',
    'Threeseasonporch',
    'ScreenPorch',
]].gt(0.0).sum(axis=1)

In [19]:
X_3.head()

,PorchTypes
0,2
1,2
2,2
3,0
4,2


In [20]:
df['MSSubClass'].unique()

array(['One_Story_1946_and_Newer_All_Styles', 'Two_Story_1946_and_Newer',
       'One_Story_PUD_1946_and_Newer',
       'One_and_Half_Story_Finished_All_Ages', 'Split_Foyer',
       'Two_Story_PUD_1946_and_Newer', 'Split_or_Multilevel',
       'One_Story_1945_and_Older', 'Duplex_All_Styles_and_Ages',
       'Two_Family_conversion_All_Styles_and_Ages',
       'One_and_Half_Story_Unfinished_All_Ages',
       'Two_Story_1945_and_Older', 'Two_and_Half_Story_All_Ages',
       'One_Story_with_Finished_Attic_All_Ages',
       'PUD_Multilevel_Split_Level_Foyer',
       'One_and_Half_Story_PUD_All_Ages'], dtype=object)

In [22]:
X_4 = pd.DataFrame()
X_4['MSClass'] = df['MSSubClass'].str.split("_", n = 1, expand=True)[0]
X_4.head()

,MSClass
0,One
1,One
2,One
3,One
4,Two


In [23]:
X_5 = pd.DataFrame()
X_5['MedNhbdArea'] = df.groupby("Neighborhood")["GrLivArea"].transform("median")
X_5.head()

,MedNhbdArea
0,1200.0
1,1200.0
2,1200.0
3,1200.0
4,1560.0


In [27]:
feature = ['MedNhbdArea','Neighborhood','GrLivArea']
X_6 = df.reindex(columns=feature)


In [28]:
X_6.head()

,MedNhbdArea,Neighborhood,GrLivArea
0,NaN,North_Ames,1656.0
1,NaN,North_Ames,896.0
2,NaN,North_Ames,1329.0
3,NaN,North_Ames,2110.0
4,NaN,Gilbert,1629.0


In [33]:
X_new = X.join([X_1, X_2, X_3, X_4, X_5])
score_dataset(X_new, y)

0.1384541031081946